# NDN-DPDK Fileserver 

## Configure Fabric Environment

In [4]:
import os

os.environ['FABRIC_PROJECT_ID']='d767bb56-903c-44e3-84a9-9aaa39acff98'
os.environ['FABRIC_BASTION_HOST'] = 'bastion-1.fabric-testbed.net'
os.environ['FABRIC_BASTION_USERNAME']='sankalpatimilsina12_0049769586'
os.environ['FABRIC_BASTION_KEY_LOCATION']=os.environ['HOME']+'/work/.ssh/fabric_bastion_key'
os.environ['FABRIC_SLICE_PRIVATE_KEY_FILE']=os.environ['HOME']+'/.ssh/id_rsa'
os.environ['FABRIC_SLICE_PUBLIC_KEY_FILE']=os.environ['HOME']+'/.ssh/id_rsa.pub'

### Import FABRIC API

In [6]:
import json
import time

from fabrictestbed_extensions.fablib.fablib import fablib

## Configure Slice Parameters



In [ ]:
try:
    #sliceName = 'fileserver'
    sliceNum = int(time.time())
    sliceName = f'fileserver@{sliceNum}'
    print(sliceName)
    
    # FABRIC site to allocate slice; it must have IPv4 to reach the router
    SITE = 'MASS'
    # remote router on /ndn network, written as IPv4 address (not hostname) and UDP port
    ROUTER_IP, ROUTER_PORT = '128.196.203.36', 6363
    # URI for NDNts-CA profile packet, base64-encoded; the CA must accept "nop" challenge
    CA_PROFILE_B64_URI = 'https://gist.githubusercontent.com/yoursunny/54db5b27f9193859b7d1c83f0aeb8d6d/raw/ca-profile.data.base64'
    # NDN names for intermediate certificates
    CA_INTERMEDIATE_NAMES = [
        '/ndn/nl/delft/%40GUEST/honecod163%40nifect.com/KEY/56=%00%05%DF%ED%DA%23%AA%98/NA/54=%00%00%01%81%01%81%F0%A2',
        '/ndn/nl/delft/KEY/%E9%EB%8E%9F%F7%A4%BF%3F/NA/54=%00%00%01%80%29%F7%A1%9D',
    ]
    # NDN name prefix for the file server
    FS_PREFIX = f'/fileserver.{sliceNum}'
    # filesystem path for the file server
    FS_PATH = '/srv/fileserver'
    # segment length served by the file server
    FS_SEGMENTLEN = 6*1024

    slice = fablib.new_slice(name=sliceName)
    nodeP = slice.add_node(name='P', site=SITE, cores=12,
                           ram=32, disk=80, image='default_ubuntu_22')
    diskP = nodeP.add_component(model='NVME_P4510', name='disk')
    nodeP.add_component(model='NIC_Basic', name='nic1')
    slice.submit()
    
    slice = fablib.get_slice(name=sliceName)
    nodeP = slice.get_node(name='P')
    diskP = nodeP.get_component('disk')
    print(nodeP.get_ssh_command())
    
except Exception as e:
    print(f"Slice Fail: {e}")

### Install Packages

In [ ]:
nodeP.execute(f'''
    echo "deb [arch=amd64 trusted=yes] https://nfd-nightly-apt.ndn.today/ubuntu jammy main" | sudo tee /etc/apt/sources.list.d/nfd-nightly.list
    sudo apt update
    sudo DEBIAN_FRONTEND=noninteractive apt full-upgrade -y
    sudo DEBIAN_FRONTEND=noninteractive apt install -y --no-install-recommends jq libibverbs-dev linux-image-generic ndnsec ndnpeek nfd ndn-tools
    sudo loginctl enable-linger {nodeP.get_username()}
    sudo reboot
''')
slice.wait_ssh(progress=True)
diskP.configure_nvme(mount_point=FS_PATH)

nodeP.execute('''
    git clone https://github.com/yoursunny/ndn-dpdk.git
    cd ndn-dpdk
    docs/ndndpdk-depends.sh -y

    corepack pnpm install
    make
    sudo make install
''')

### Run NFD and obtain testbed certificate

In [ ]:
import shlex

peekCmds = '\n'.join(
    [f'ndnpeek {name} | base64 > ~/keychain/i{i}.ndncert' for i, name in enumerate(CA_INTERMEDIATE_NAMES)])

stdout, stderr = nodeP.execute(f'''
    sudo systemctl enable --now nfd
    nfdc face create udp4://{ROUTER_IP}:{ROUTER_PORT}
    nfdc route add / udp4://{ROUTER_IP}:{ROUTER_PORT}
    mkdir -p ~/keychain
    {peekCmds}

    sudo npm i -g https://ndnts-nightly.ndn.today/keychain-cli.tgz
    export NDNTS_UPLINK=unix:///run/nfd.sock

    curl -fsLS {shlex.quote(CA_PROFILE_B64_URI)} | base64 -d > ~/keychain/ca-profile.data
    CAPREFIX=$(ndnts-keychain ndncert03-show-profile --profile ~/keychain/ca-profile.data | sed -n '/^CA prefix:/ s/^[^:]*: // p')
    ndnsec key-gen $CAPREFIX/$(hostname -s) >/dev/null
    KEYNAME=$(ndnsec get-default -k)
    ndnts-keychain ndncert03-client --profile ~/keychain/ca-profile.data --ndnsec --key $KEYNAME --challenge nop
    ndnsec export -o ~/keychain/pvt.safebag -P 0 -k $KEYNAME

    sudo systemctl disable --now nfd
    
    to_file() {{
        sudo mkdir -p "$(dirname $1)"
        sudo tee "$1"
    }}
    echo -e '[Manager]\nCPUAffinity=0-1' | to_file /etc/systemd/system.conf.d/cpuset.conf
    echo -e '[Service]\nCPUAffinity=2-7' | to_file /etc/systemd/system/ndndpdk-svc@$(systemd-escape 127.0.0.1:3030).service.d/override.conf
    echo -e '[Service]\nCPUAffinity=8-11' | to_file /etc/systemd/system/ndndpdk-svc@$(systemd-escape 127.0.0.1:3031).service.d/override.conf
    sudo reboot
''')
slice.wait_ssh(progress=True)


### Set NDN-DPDK Forwarder args and NFD registration command

In [12]:
FS_PREFIX = f'/fileserver.{sliceNum}'

FW_ACTIVATE = {
    'eal': {
        'memPerNuma': {'0': 12*1024},
        'filePrefix': 'fw',
        'disablePCI': True,
    },
    'mempool': {
        'DIRECT': {'capacity': 2**20-1, 'dataroom': 9200},
        'INDIRECT': {'capacity': 2**21-1},
    },
}
FW_UDP = {
    'scheme': 'udp',
    'remote': f'{ROUTER_IP}:{ROUTER_PORT}',
    'mtu': 1420,
}
MEMIF_SOCKET, MEMIF_ID = '/run/ndn/fileserver.sock', 1
FW_MEMIF = {
    'scheme': 'memif',
    'socketName': MEMIF_SOCKET,
    'id': MEMIF_ID,
    'role': 'server',
}
FS_ACTIVATE = {
    'eal': {
        'memPerNuma': {'0': 6*1024},
        'filePrefix': 'fs',
        'disablePCI': True,
    },
    'mempool': {
        'DIRECT': {'capacity': 2**16-1, 'dataroom': 9200},
        'INDIRECT': {'capacity': 2**16-1},
        'PAYLOAD': {'capacity': 2**16-1, 'dataroom': 9200},
    },
    'face': {
        'scheme': 'memif',
        'socketName': MEMIF_SOCKET,
        'id': MEMIF_ID,
        'role': 'client',
    },
    'fileServer': {
        'mounts': [
            {'prefix': FS_PREFIX, 'path': FS_PATH}
        ],
        'segmentLen': FS_SEGMENTLEN,
    },
}

nfdregCmd = 'ndndpdk-godemo --gqlserver http://127.0.0.1:3030/ nfdreg'
nfdregCmd += ' --signer ~/keychain/pvt.safebag --signer-pass 0 '
nfdregCmd += ' '.join([f'--serve-cert ~/keychain/i{i}.ndncert' for i,
                      name in enumerate(CA_INTERMEDIATE_NAMES)])
nfdregCmd += f' --origin 65 --register {shlex.quote(FS_PREFIX)} --repeat 20s'

### Run NDN-DPDK Forwarder, Fileserver and Periodically send prefix registration command to remote NFD

In [ ]:
stdout, stderr = nodeP.execute(f'''
    sudo mkdir -p {shlex.quote(FS_PATH)}
    sudo mount /dev/nvme0n1p1 {shlex.quote(FS_PATH)}
    sudo chown {nodeP.get_username()} {shlex.quote(FS_PATH)}
    truncate -s 1G {shlex.quote(f'{FS_PATH}/1G.bin')}
    truncate -s 1M {shlex.quote(f'{FS_PATH}/1M.bin')}

    CTRL_FW='ndndpdk-ctrl --gqlserver http://127.0.0.1:3030/'
    CTRL_FS='ndndpdk-ctrl --gqlserver http://127.0.0.1:3031/'
    systemctl --user stop nfdreg || true
    sudo $CTRL_FW systemd stop || true
    sudo $CTRL_FS systemd stop || true

    sudo dpdk-hugepages.py --clear
    sudo dpdk-hugepages.py --pagesize 1G --setup 18G

    sudo $CTRL_FW systemd start
    echo {shlex.quote(json.dumps(FW_ACTIVATE))} | $CTRL_FW activate-forwarder

    FW_UDP_FACE=$(echo {shlex.quote(json.dumps(FW_UDP))} | $CTRL_FW create-face)
    echo $FW_UDP_FACE
    $CTRL_FW insert-fib --name /localhop/nfd --nh $(echo $FW_UDP_FACE | jq -r .id)
    
    systemd-run --user --collect --unit=nfdreg -- {nfdregCmd}

    FW_MEMIF_FACE=$(echo {shlex.quote(json.dumps(FW_MEMIF))} | $CTRL_FW create-face)
    echo $FW_MEMIF_FACE
    $CTRL_FW insert-fib --name {shlex.quote(FS_PREFIX)} --nh $(echo $FW_MEMIF_FACE | jq -r .id)

    sudo $CTRL_FS systemd start
    echo {shlex.quote(json.dumps(FS_ACTIVATE))} | $CTRL_FS activate-fileserver
    
    $CTRL_FW list-faces
    $CTRL_FW list-fib
''')
print(f'{stdout}\n{stderr}')

## Delete Slice

In [ ]:
try:
    slice = fablib.get_slice(name=sliceName)
    slice.delete()
except Exception as e:
    print(f"Fail: {e}")